In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sbi_ice.simulators import Layer_Tracing_Sim as lts
from tueplots import figsizes
from sbi_ice.utils import plotting_utils,misc
data_dir,output_dir = misc.get_data_output_dirs()

color_opts = plotting_utils.setup_plots()

# Decoupling Mass Balance Figure

Here we create Fig. 1 of the publication.

In [ ]:
#We create 3 different SMB/BMB combinations for the same shelf and simulate them all.

shelf = "Toy"
data_file = Path(data_dir,shelf,"setup_files","flowtube_3d_complete.csv")
df = pd.read_csv(data_file)
xmb = df['x_coord'].to_numpy() #x - coordinates of domain

nx_iso = 300
ny_iso = 1
time_init          = 0   # [yr] Starting time
dt                 = 0.5 # [yr] timestep


smb = df['smb'].to_numpy()
bmb = df['bmb'].to_numpy()
bump = smb - 0.3
bmb_const = np.ones(bmb.shape[0])
smb_const = 0.3*np.ones(bmb.shape[0])

smb1 = smb_const
bmb1 = bmb_const

smb2 = smb.copy()
bmb2 = bmb_const

smb3 = smb_const - bump/2
bmb3 = bmb_const + bump

n_surface_phase1 = 10
n_base_phase1 = 10
time_phase1 = 2000
sched1 = lts.Scheduele(time_phase1,dt,n_surface_phase1,n_base_phase1)


geom1 = lts.Geom(nx_iso=nx_iso,ny_iso=ny_iso)
smb_regrid1,bmb_regrid1 = lts.init_geom_from_fname(geom1,data_file,smb=smb1)
geom1.initialize_layers(sched1,10)
lts.sim(geom1,smb_regrid1,bmb_regrid1,sched1)


geom2 = lts.Geom(nx_iso=nx_iso,ny_iso=ny_iso)
smb_regrid2,bmb_regrid2 = lts.init_geom_from_fname(geom2,data_file,smb=smb2)
geom2.initialize_layers(sched1,10)
lts.sim(geom2,smb_regrid2,bmb_regrid2,sched1)

geom3 = lts.Geom(nx_iso=nx_iso,ny_iso=ny_iso)
smb_regrid3,bmb_regrid3 = lts.init_geom_from_fname(geom3,data_file,smb=smb3)
geom3.initialize_layers(sched1,10)
lts.sim(geom3,smb_regrid3,bmb_regrid3,sched1)


In [ ]:
"""
Plot Figure 1:

We plot the shelf.
We split the plot into two parts, where we demonstrate using the traditional "mass budget" method of getting total mass balance.
In the second part, we show our method of using the layer depths to separate into surface and basal mass balance.
"""
ages = [100,500]
age_indices = []
midway = int(geom1.age_iso.size/2)
for age in ages:
    age_indices.append(midway + np.argmin(np.abs(geom1.age_iso[midway:]-age)))
offsets = [-12,-10]

plt.rcParams.update(figsizes.icml2022_full(nrows=1,height_to_width_ratio=1.0))

fig = plt.figure()


xdom = geom1.x/1000 #work in km
xlim = 15 #splitting into two mass balance methods
lmask = xdom > xlim #mask for right side of plot
narrrows = 5 #number of arrows for the inflow boundary
tmbbounds = [5,12] #boundaries of mass budget method
bidxs = [np.argmin(np.abs(xdom-tmbbounds[0])),np.argmin(np.abs(xdom-tmbbounds[1]))] #boundaries in geometry indices
tmbmid = np.mean(tmbbounds) #midpoint of mass budget method
midx = np.argmin(np.abs(xdom-tmbmid)) #midpoint in geometry indices
linespacing = 50 #spacing between lines in different annotations

ax_cutoff =  0.1+0.85 * xlim/xdom[-1] #location of inset axes for smb and bmb
ax_width = 0.85 - ax_cutoff


sample1_color = "#526d7e"
sample2_color = "#975300"
sample3_color = "#007920"
annotation_color = "#1c4585"


"""
ax1 contains the main plot of the shelf and layers
"""
ax1 = fig.add_axes([0.1, 0.1, 0.85, 0.85])
#plot shelf outlines
ax1.plot(xdom,geom1.ss[:,0],color="black",label="Surface",linewidth=1.5,zorder=10)
ax1.plot(xdom,geom1.bs[:,0],color="black",label="Bed",linewidth=1.5,zorder=10)
ax1.fill_between(xdom,geom1.ss[:,0],geom1.bs[:,0],color="black",linewidth=0.0,alpha=0.075,label="Shelf")

#Plot inflow direction
arrow_zs = np.linspace(geom1.bs[0,0]+50,geom1.ss[0,0]-50,narrrows)
divide_z = np.mean(arrow_zs[1:3])
for i in range(narrrows):
    ax1.arrow(xdom[0]-1,arrow_zs[i],1.5,0,head_width=5,head_length=0.2,color=annotation_color,linewidth=1.0)
#ax1.annotate("inflow",xy=(xdom[0],arrow_zs[-1]),xycoords="data",textcoords="offset points",xytext=(15,-5),ha="center",va="top",color="C5")

#Annotate mass budget method in left side of domain
ax1.vlines(tmbbounds,geom1.bs[bidxs,0],geom1.ss[bidxs,0],color= annotation_color)
for i,idx in enumerate(bidxs):
    ax1.arrow(xdom[idx]-1.5,-100,1,0,head_width=5,head_length=0.2,color ="black")
    ax1.annotate( r"${v_{" + str(i+1) + "}}h_{" + str(i+1) + "}$", xy=(xdom[idx]-2.0, -100), xytext=(-2, 4),xycoords="data",textcoords="offset points",color="black")
    #ax.annotate(r"$\mathbf{v_{1}}H_{1}$", xy=(xdom[idx]-0.5, -100), xytext=(-10, 0),arrowprops=dict(arrowstyle="->"),xycoords="data",textcoords="offset points")

ax1.arrow(tmbmid-0.2, geom1.ss[midx,0]+10,0,-20,head_width=0.2,head_length=5,color="black")
ax1.arrow(tmbmid+0.2, geom1.ss[midx,0]-10,0,20,head_width=0.2,head_length=5,color="black")
ax1.annotate(r"$\dot{a}$",xy=(tmbmid-0.1,geom1.ss[midx,0]+10),xycoords="data",textcoords="offset points",xytext=(0,10),ha="center",va="top",color="black")
ax1.arrow(tmbmid-0.2, geom1.bs[midx,0]+10,0,-20,head_width=0.2,head_length=5,color="black")
ax1.arrow(tmbmid+0.2, geom1.bs[midx,0]-10,0,20,head_width=0.2,head_length=5,color="black")
ax1.annotate(r"$\dot{b}$",xy=(tmbmid-0.15,geom1.bs[midx,0]+10),xycoords="data",textcoords="offset points",xytext=(0,10),ha="center",va="top",color="black")

#Annotate Grounding Line
ax1.vlines(xdom[0],geom1.bs[0,0]-20,geom1.ss[0,0]+20,color="black",linestyle="dotted")
ax1.annotate("GL",xy=(xdom[0],geom1.bs[0,0]-50),xycoords="data",textcoords="offset points",xytext=(0,0),ha="center",va="bottom",color="black")
ax1.annotate("Ice \n Sheet",xy = (xdom[0]-1.6,divide_z),xycoords="data",textcoords="offset points",xytext=(0,0),ha="center",va="center",color="black")
ax1.annotate("Ice \n Shelf",xy = (xdom[0]+1.5,divide_z),xycoords="data",textcoords="offset points",xytext=(0,0),ha="center",va="center",color="black")



#Annotate Steady State
ax1.annotate("Steady State",xy=(0.025,0.95),xycoords="axes fraction",textcoords="offset points",xytext=(0,0),ha="left",va="top",color = "black")

#Describe Mass Budget Method
ax1.annotate("Mass Budget \n method",xy=(tmbmid,geom1.ss[midx,0]+10+linespacing*1.6),xycoords="data",textcoords="offset points",xytext=(0,0),ha="center",va="top",color=annotation_color)
ax1.annotate("Contemporary \n estimates",xy=(tmbmid,geom1.bs[bidxs[0],0]-40),xycoords="data",textcoords="offset points",xytext=(0,0),ha="center",va="top",color="black")

ax1.annotate(r"$\dot{b}_{est} = \dot{a}_{obs} - \mathbf{\nabla}\cdot(vh)$",xy=(tmbmid,geom1.bs[bidxs[0],0]-40-linespacing),xycoords="data",textcoords="offset points",xytext=(0,0),ha="center",va="top",color="black")

#Describe IRH Method
ax1.annotate('Internal Reflection Horizon (IRH) method',xy=(25,-230),xycoords="data",textcoords="offset points",xytext=(0,0),ha="center",va="top",color=annotation_color)
ax1.annotate('\n time-averaged estimates $\dot{a}_{avg},\dot{b}_{avg}$',xy=(25,-230),xycoords="data",textcoords="offset points",xytext=(0,0),ha="center",va="top",color="black")
#Plot Layers in right side of domain

for i,age in enumerate(age_indices):
    ax1.plot(xdom[lmask],geom1.bs[lmask,0]+geom1.dsum_iso[lmask,0,age],color=sample1_color,linestyle="dashed",linewidth=1.0)
    ax1.plot(xdom[lmask],geom1.bs[lmask,0]+geom2.dsum_iso[lmask,0,age],color=sample2_color,linestyle="dashed",linewidth=1.0)
    ax1.plot(xdom[lmask],geom1.bs[lmask,0]+geom3.dsum_iso[lmask,0,age],color=sample3_color,linestyle="dashed",linewidth=1.0)

    ax1.annotate("age = {:.0f}a".format(geom1.age_iso[age]),xy=(xdom[lmask][0],geom1.bs[lmask,0][0]+geom1.dsum_iso[lmask,0,age][0]),xycoords="data",textcoords="offset points",xytext=(0,offsets[i]))
    

ax1.set_ylabel("Elevation [m.a.s.l.]")
ax1.set_ylim(np.min(geom1.bs)-100,np.max(geom1.ss)+100)
ax1.set_xlabel("Distance [km]")
ax1.spines['bottom'].set_bounds(xdom[0]-0.001,xdom[-1]+1)

"""
SMB inset
"""

ax2 = fig.add_axes([ax_cutoff, 0.8, ax_width, 0.12])
ax2.plot(xdom[lmask],smb_regrid1[lmask,0],color=sample1_color,linewidth=1.0)
ax2.plot(xdom[lmask],smb_regrid2[lmask,0],color=sample2_color,linewidth=1.0)
ax2.plot(xdom[lmask],smb_regrid3[lmask,0],color=sample3_color,linewidth=1.0)

"""
Move spines to right side for clarity
"""
ax2.spines['bottom'].set_visible(False)
ax2.tick_params(bottom=False)
ax2.axes.xaxis.set_ticklabels([])
ax2.set_ylabel(r"$\dot{a}$ [m/a]",labelpad=2.5,rotation=90)
ax2.yaxis.set_label_position("right")
ax2.spines[['left']].set_visible(False)
ax2.spines[['right']].set_visible(True)
ax2.yaxis.tick_right()

"""
BMB inset
"""

ax3 = fig.add_axes([ax_cutoff, 0.15, ax_width, 0.15])
ax3.plot(xdom[lmask],-bmb_regrid1[lmask,0],color=sample1_color,linewidth=1.0)
ax3.plot(xdom[lmask],-bmb_regrid2[lmask,0],color=sample2_color,linewidth=1.0)
ax3.plot(xdom[lmask],-bmb_regrid3[lmask,0],color=sample3_color,linewidth=1.0)


ax3.spines['bottom'].set_visible(False)
ax3.tick_params(bottom=False)
ax3.axes.xaxis.set_ticklabels([])
ax3.set_ylabel(r"$\dot{b}$ [m/a]",labelpad=2.5,rotation=90)
ax3.yaxis.set_label_position("right")
ax3.spines[['left']].set_visible(False)
ax3.spines[['right']].set_visible(True)
ax3.yaxis.tick_right()


#Save figure
fig_name = Path(output_dir,"paper_figures","Decoupling_MB","Decoupling_MB.pdf")
fig_name.parent.mkdir(parents=True,exist_ok=True)
fig.savefig(fig_name)



In [ ]:
print(plt.rcParams)